<a href="https://colab.research.google.com/github/shinome551/colab/blob/master/NN_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
import numpy as onp
from sklearn.datasets import fetch_openml

In [0]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X / 255.0
y = y.astype(onp.uint8)

In [0]:
key = random.PRNGKey(0)

In [0]:
X_perm = random.permutation(key, X)
y_perm = random.permutation(key, y)
X_train, X_test = np.split(X_perm, [60000])
y_train, y_test = np.split(y_perm, [60000])

In [0]:
@jit
def dist_vv(x, y):
    return np.vdot(x, x.T) - 2 * np.vdot(x, y.T) + np.vdot(y, y.T)

dist_vm = vmap(dist_vv, (None, 0), 0)

In [0]:
@jit
def NN_vm(q, X):
    d = dist_vm(q, X)
    return np.argmin(d, axis=0)

NN_mm = vmap(NN_vm, (0, None), 1)

In [0]:
pred = y_train[NN_mm(X_test, X_train)]

In [27]:
print('Test Accuracy:{:.1f}%'.format(100 * (y_test == pred).mean()))

Test Accuracy:97.3%
